# 2. run RNN on affvids: 

*Yiyu Wang 2022 October*

In [1]:
%conda env list

# conda environments:
#
HTFATorch                /home/wang.yiyu/.conda/envs/HTFATorch
NTFA_env3                /home/wang.yiyu/.conda/envs/NTFA_env3
base                  *  /work/abslab/Yiyu/DNN_env


Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import math
import time
import glob
import random
import string
import collections

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import KFold

from tqdm import tqdm


SEED = 2022
N_ROI = 100

In [3]:
# load the subjects:
included_data = pd.read_csv('/work/abslab/AVFP/Preproc_Scripts/included_AVFP_novel_subjects.csv', header=None)
subIDs = included_data[0].astype('str').tolist()


## Set up data:

In [4]:

class CustomAffVidsDynamicDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        return X, y


In [5]:


def GetDataName(subject, run='*'):
    file_name = glob.glob(f'/work/abslab/Yiyu/dnn/AVFP_parcellation/wholebrain_schaeffer_{N_ROI}/par-{subject}_run-{run}_schaefer_{N_ROI}.csv')
    if not file_name :
        raise Exception('No this parcellation csv file!')
    return file_name

def CreateXY(sub_list, network = None, y_col='fear'):
    """
    Run gradient descent to opimize parameters of a given network

    Args:
    sub_list: list
        list of subject IDs to extract the data
    network: str 
        which network to lesion, or default is no lesion 
        the network name must be in the dataframe column names
    y_col: str
        which column to extract the y variable
        default = 'fear'

    Returns:
    X_tensor: Tensor
        batch x seq x feature
    Y_tensor: Tensor
    """
    
    X = []
    Y = []
    for sub in sub_list:
        for parcellation_path in GetDataName(sub):
            par_df =pd.read_csv(parcellation_path)
            par_df = par_df.loc[par_df['video_name']!='0']
            
            if network:
                x_cols = [col for col in par_df.columns if 'Networks' in col]
                zero_cols = [col for col in x_cols if (network in col)]
                par_df[zero_cols]=0
                  
            avg = par_df[y_col].mean()
            std = par_df[y_col].std()
            for vid_name, trial_df in par_df.groupby('video_name'):
                if ~np.isnan(trial_df[y_col].unique()[0]):
                    X.append(trial_df.iloc[:,0:N_ROI].astype(float).values)
                    Y.append((trial_df[y_col].unique()[0]-avg)/std)

    # concate the x and y
    # x: batch x seq  x feature
    X_tensor = torch.tensor(np.array(X))
    # .permute(2, 1, 0)
    Y_tensor = torch.tensor(Y)
    return X_tensor, Y_tensor
     
    
    

## Set up the RNN model (LSTM):

In [32]:
class BrainLTSM_Classifier(nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, feature_dim, n_layers):
        super(BrainLTSM_Classifier, self).__init__()
        
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.feature_dim = feature_dim #input dim
        self.n_layers = n_layers
        self.batch_size = batch_size
        
        self.init_linear = nn.Linear(self.feature_dim, self.feature_dim)

        self.lstm = nn.LSTM(feature_dim, hidden_size, n_layers, batch_first=True,dropout=.2, bidirectional=True)
        
        # Define the output layer
        self.linear = nn.Linear(self.hidden_size * 2, output_size) #bidirectional: hidden_size * 2 (for backward processing)
        # self.linear = nn.Linear(self.hidden_size, output_size) #unidirectional
      

    def init_hidden(self):
        # batch = len(inputs) (if batch_first = True)
        return (torch.zeros(self.n_layers, self.batch_size, self.hidden_size),
                torch.zeros(self.n_layers, self.batch_size, self.hidden_size))

    def forward(self, input):
        #Forward pass through initial hidden layer
        linear_input = self.init_linear(input)

        # Forward pass through LSTM layer
        # shape of lstm_out: [batch_size, input_size ,hidden_dim]
        # shape of self.hidden: (a, b), where a and b both
        # have shape (batch_size, num_layers, hidden_dim).
        lstm_out, self.hidden = self.lstm(linear_input)

        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out)
        return y_pred, self.hidden    

In [33]:
# model training functions:

def GetMaxAcc(max_acc, current_acc):
    save_state = False
    if abs(current_acc) > abs(max_acc):
        max_acc = current_acc
        save_state = True
    return max_acc, save_state

def EpochTrainTest(model, loss_fn, train_loader, test_loader,
          n_epochs, optimizer, device='cpu', verbose=False):
    """
    Run gradient descent to opimize parameters of a given model

    Args:
    model: nn.Module
      PyTorch network whose parameters to optimize
    loss_fn:
      loss function to minimize
    train_loader: Dataloader object
    test_loader: Dataloader object
    n_epoch: Integer
      number of epochs
    optimizer: 
      built-in optimizer from torch


    Returns:
    train_loss/test_loss: List
      Training/Test loss over epochs
    """

    # Placeholder to save the loss at each iteration
    train_loss = []
    test_loss = []
    train_acc = []
    test_acc = []
    max_acc = 0
    
    for epoch in range(n_epochs):
        epoch_train_loss = []
        epoch_train_accuracy = []
        
        # Train:
        model.train()
        for X, Y in tqdm(train_dataloader):
            X.to(device)
            Y.to(device)

            Y_preds, hidden = model(X.float())

            this_loss = loss_fn(Y_preds[:,-1,0], Y.float())
            epoch_train_loss.append(this_loss.item())
            # Clear previous gradients
            optimizer.zero_grad()
            # Compute gradients
            this_loss.backward()
            # Update weights
            optimizer.step()
            
            this_acc= np.corrcoef(Y.detach().numpy(),
                                      Y_preds.detach().squeeze().numpy()[:,-1])[0,1]
            epoch_train_accuracy.append(this_acc)
            
        train_loss.append(torch.tensor(epoch_train_loss).mean())
        train_acc.append(torch.tensor(epoch_train_accuracy).mean())
         
        # Test:
        model.eval()  
        with torch.no_grad():
            Y_shuffled, Y_preds = [],[]
            for X, Y in test_dataloader:
                X.to(device),Y.to(device)
                preds, hidden = model(X.float())
                epoch_test_loss = loss_fn(preds[:,-1,0], Y.float())
                
                test_loss.append(epoch_test_loss.item())
                Y_shuffled.append(Y)
                Y_preds.append(preds)
            Y_shuffled = torch.cat(Y_shuffled)
            Y_preds = torch.cat(Y_preds)
            epoch_test_accuracy = np.corrcoef(Y_shuffled.detach().numpy(),
                                          Y_preds.detach().squeeze().numpy()[:,-1])[0,1]  
            
            test_acc.append(epoch_test_accuracy)
            max_acc, save_state = GetMaxAcc(max_acc, epoch_test_accuracy)
            if save_state:
                model_state = model.state_dict()
                


        if verbose:
            print("Train Loss : {:.3f}".format(torch.tensor(epoch_train_loss).mean()))   
            print("Train accuracy {:.3f}".format(torch.tensor(epoch_train_accuracy).mean()))
            print(f'iteration {epoch + 1}/{n_epochs} | train loss: {np.mean(epoch_train_loss):.3f} | train acc: {np.mean(epoch_train_accuracy):.3f}')
            print(f'iteration {epoch + 1}/{n_epochs} | test loss: {epoch_test_loss:.3f} | test acc: {epoch_test_accuracy:.3f}')

  
    print('maximum accuracy: {:.3f}'.format(max_acc))
    print('mean accuracy: {:.3f}'.format(np.mean(test_acc)))
    
    

    return  max_acc, model_state

## Run training and validation:

In [34]:
# run kfold on subjects:
kf = KFold(n_splits=5, shuffle=True, random_state = SEED)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
max_acc_list = []
batch_size = 32
epochs = 10
learning_rate = 1e-3
loss_fn = nn.MSELoss()

# model params:
output_size = 1
hidden_size = 150
feature_dim = N_ROI
n_layers=2

for i, (train_idx, test_idx) in enumerate(kf.split(subIDs)):
    print(f'---------------------- fold {i+1} ---------------------------')
    
    sub_train = np.array(subIDs)[train_idx.astype(int)].tolist()
    sub_test = np.array(subIDs)[test_idx.astype(int)].tolist()
    
    train_X, train_Y = CreateXY(sub_train)
    test_X, test_Y = CreateXY(sub_test)

    train_dataset = CustomAffVidsDynamicDataset(train_X, train_Y)
    test_dataset = CustomAffVidsDynamicDataset(test_X, test_Y)

    train_dataloader = DataLoader(train_dataset,batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)
    
    
    model = BrainLTSM_Classifier(batch_size, output_size, hidden_size, feature_dim, n_layers)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    max_acc_fold, model_state = EpochTrainTest(model, loss_fn, train_dataloader, test_dataloader, epochs, optimizer, device)
    max_acc_list.append(max_acc_fold)
    
    # save the model_sate
    model_file_path = f'models/LSTM_feature-{feature_dim}_hidden-{hidden_size}_layers-{n_layers}_fold-{i}_lr-{learning_rate}.pt'
    print(f'saving: {model_file_path}')
    torch.save(model_state, model_file_path)
    
print('\n\n mean acc',np.mean(max_acc_list))

---------------------- fold 1 ---------------------------


100%|██████████| 101/101 [00:05<00:00, 17.62it/s]


maximum accuracy: 0.364
mean accuracy: 0.351
saving: models/LSTM_feature-100_hidden-150_layers-2_fold-0_lr-0.001.pt
---------------------- fold 2 ---------------------------


100%|██████████| 102/102 [00:05<00:00, 17.22it/s]


maximum accuracy: 0.409
mean accuracy: 0.381
saving: models/LSTM_feature-100_hidden-150_layers-2_fold-1_lr-0.001.pt
---------------------- fold 3 ---------------------------


...

maximum accuracy: 0.394
mean accuracy: 0.364
saving: models/LSTM_feature-100_hidden-150_layers-2_fold-2_lr-0.001.pt
---------------------- fold 4 ---------------------------


100%|██████████| 101/101 [00:05<00:00, 17.24it/s]


maximum accuracy: 0.288
mean accuracy: 0.269
saving: models/LSTM_feature-100_hidden-150_layers-2_fold-3_lr-0.001.pt
---------------------- fold 5 ---------------------------


100%|██████████| 102/102 [00:05<00:00, 17.16it/s]


maximum accuracy: 0.353
mean accuracy: 0.319
saving: models/LSTM_feature-100_hidden-150_layers-2_fold-4_lr-0.001.pt


 mean acc 0.36152574801743204
